In [ ]:
class CancerDataset(torch.utils.data.Dataset):
    def __init__(self, split_dataset, transforms=None, name="train"):
        self.split_dataset = split_dataset
        self.name = name
        self.transforms = transforms

        self.class_names = {
            "TUMOUR": 1,
            "NOTUMOUR": 2,
        }

        self.add_random_bbox()
        self.idx2class = {self.class_names[name]: name for name in self.class_names}
        self.img_dict = {img_idx:{"path":  img_path,
                                  "class": self.class_names[self.split_dataset[img_path]['class_name']],
                                  "bbox":  self.split_dataset[img_path]['bbox']}
                          for img_idx, img_path in enumerate(self.split_dataset)}

    def __getitem__(self, img_idx):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly.
        """
        """FILL HERE
        Create the image, label and bbox variables using the implemented img_dict dictionary.
        """
        image = self.load_image(img_idx)
        class_name = self.img_dict[img_idx]["class"]
        bbox = self.img_dict[img_idx]["bbox"]

        # Compute the area of the bounding box
        x1, y1, x2, y2 = bbox
        area = (x2 - x1) * (y2 - y1)

        # Convert everything to tensor
        img_idx = torch.tensor([img_idx])
        bbox = torch.as_tensor([bbox], dtype=torch.float32)
        class_name = torch.as_tensor([class_name], dtype=torch.int64)
        area = torch.as_tensor([area], dtype=torch.float32)

        # Use the COCO template for targets to be able to evaluate the model with COCO API
        target = {"boxes": bbox,
                  "labels": class_name,
                  "image_id": img_idx,
                  "area": area,
                  "iscrowd": torch.as_tensor([0], dtype=torch.int64)}

        # Important line! don't forget to add this
        if self.transforms:
            image, target = self.transforms(image, target)
        # return the image, the boxlist and the idx in your dataset
        return image, target

    def __len__(self):
        return len(self.split_dataset)



    def add_random_bbox(self):
        # Add a random bounding box for all images that are NORMAL.
        for img_path in self.split_dataset:
            if self.split_dataset[img_path]['class_name'] == "NOTUMOUR":

                # where is there something in the mammogram ?
                img = imageio.imread(img_path)

                # Define random radius that should not be bigger than image
                radius = np.random.randint(10, 70)

                # Set borders at zero to avoid having a bounding box that is outside the mammogram
                new_img = np.zeros(img.shape)
                new_img[radius:-radius, radius:-radius] = img[radius:-radius, radius:-radius]
                mask = new_img > 50
                mask_id_x, mask_id_y = np.where(mask)

                random_id_x, random_id_y = np.random.randint(len(mask_id_x)), np.random.randint(len(mask_id_y))
                center_x, center_y = mask_id_x[random_id_x], mask_id_y[random_id_y]
                bbox = [center_x-radius, center_y-radius, center_x+radius, center_y+radius]
                self.split_dataset[img_path]['bbox'] = bbox



    def load_image(self, img_idx):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly.
        """
        img_path = self.img_dict[img_idx]['path']
        image = imageio.imread(img_path)[..., np.newaxis]
        image = np.concatenate((image, image, image), axis=2)
        image = Image.fromarray(image).convert("RGB")
        return image


running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/mask.py -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/_mask.cpython-37m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/coco.py -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/__init__.py -> build/bdist.lin

fatal: destination path 'cocoapi' already exists and is not an empty directory.
zip_safe flag not set; analyzing archive contents...
pycocotools.__pycache__._mask.cpython-37: module references __file__
fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at be376084 version check against PyTorch's CUDA version
